In [1]:
import pandas as pd
import numpy as np
from scipy import stats
%matplotlib inline
import matplotlib.pyplot as plt
import math

from collections import defaultdict
from geoalchemy2.shape import to_shape
from geoalchemy2.elements import WKTElement
from sqlalchemy import *

In [3]:
PG_USER = 'nadai'
PG_DBNAME = 'WWW'

# Creating SQLAlchemy's engine to use
engine = create_engine('postgresql://{user}@localhost:5432/{dbname}'.format(user=PG_USER, dbname=PG_DBNAME))

In [4]:
quarters = defaultdict(list)

sql = text('SELECT i.pro_com::int, i.ace::int, c.name FROM istat_sezioni i INNER JOIN cities c ON i.pro_com = c.pro_com WHERE ace <> 0')
result = engine.execute(sql)
names = []
CITIES = {}
for r in result:
    quarters[r[0]].append(r[1])
    CITIES[r[2]] = r[0]

# Features computation

In [6]:
age_intervals = [
    [1919, 1945],
    [1946, 1960],
    [1961, 1970],
    [1971, 1980],
    [1981, 1990],
    [1991, 2000],
    [2001, 2010],
]
age_mean_intervals = []
for low, high in age_intervals:
    now = age_intervals[-1][1]
    avg = float((now-low) + (now-high))/2.
    age_mean_intervals.append(avg)
age_mean_intervals = np.array(age_mean_intervals)
age_mean_intervals

array([78. , 57. , 44.5, 34.5, 24.5, 14.5,  4.5])

In [7]:
htypes_w = np.array([
    1., 2., 3., 7.
])

In [9]:
import math, random


# Data conventions: A point is a pair of floats (x, y). A circle is a triple of floats (center x, center y, radius).

# 
# Returns the smallest circle that encloses all the given points. Runs in expected O(n) time, randomized.
# Input: A sequence of pairs of floats or ints, e.g. [(0,5), (3.1,-2.7)].
# Output: A triple of floats representing a circle.
# Note: If 0 points are given, None is returned. If 1 point is given, a circle of radius 0 is returned.
# 
def make_circle(points):
    # Convert to float and randomize order
    shuffled = [(float(p[0]), float(p[1])) for p in points]
    random.shuffle(shuffled)
    
    # Progressively add points to circle or recompute circle
    c = None
    for (i, p) in enumerate(shuffled):
        if c is None or not _is_in_circle(c, p):
            c = _make_circle_one_point(shuffled[0 : i + 1], p)
    return c


# One boundary point known
def _make_circle_one_point(points, p):
    c = (p[0], p[1], 0.0)
    for (i, q) in enumerate(points):
        if not _is_in_circle(c, q):
            if c[2] == 0.0:
                c = _make_diameter(p, q)
            else:
                c = _make_circle_two_points(points[0 : i + 1], p, q)
    return c


# Two boundary points known
def _make_circle_two_points(points, p, q):
    diameter = _make_diameter(p, q)
    if all(_is_in_circle(diameter, r) for r in points):
        return diameter
    
    left = None
    right = None
    for r in points:
        cross = _cross_product(p[0], p[1], q[0], q[1], r[0], r[1])
        c = _make_circumcircle(p, q, r)
        if c is None:
            continue
        elif cross > 0.0 and (left is None or _cross_product(p[0], p[1], q[0], q[1], c[0], c[1]) > _cross_product(p[0], p[1], q[0], q[1], left[0], left[1])):
            left = c
        elif cross < 0.0 and (right is None or _cross_product(p[0], p[1], q[0], q[1], c[0], c[1]) < _cross_product(p[0], p[1], q[0], q[1], right[0], right[1])):
            right = c
    return left if (right is None or (left is not None and left[2] <= right[2])) else right


def _make_circumcircle(p0, p1, p2):
    # Mathematical algorithm from Wikipedia: Circumscribed circle
    ax = p0[0]; ay = p0[1]
    bx = p1[0]; by = p1[1]
    cx = p2[0]; cy = p2[1]
    d = (ax * (by - cy) + bx * (cy - ay) + cx * (ay - by)) * 2.0
    if d == 0.0:
        return None
    x = ((ax * ax + ay * ay) * (by - cy) + (bx * bx + by * by) * (cy - ay) + (cx * cx + cy * cy) * (ay - by)) / d
    y = ((ax * ax + ay * ay) * (cx - bx) + (bx * bx + by * by) * (ax - cx) + (cx * cx + cy * cy) * (bx - ax)) / d
    return (x, y, math.hypot(x - ax, y - ay))


def _make_diameter(p0, p1):
    return ((p0[0] + p1[0]) / 2.0, (p0[1] + p1[1]) / 2.0, math.hypot(p0[0] - p1[0], p0[1] - p1[1]) / 2.0)


_EPSILON = 1e-12

def _is_in_circle(c, p):
    return c is not None and math.hypot(p[0] - c[0], p[1] - c[1]) < c[2] + _EPSILON


# Returns twice the signed area of the triangle defined by (x0, y0), (x1, y1), (x2, y2)
def _cross_product(x0, y0, x1, y1, x2, y2):
    return (x1 - x0) * (y2 - y0) - (y1 - y0) * (x2 - x0)

In [10]:
import sys
import traceback


columns=[('a', 'b')]
results_df = pd.DataFrame({}, index=pd.MultiIndex.from_tuples(columns, names=['ace', 'pro_com']))
#clean the dataframe
results_df = results_df[results_df.index.get_level_values('ace') == 'Ras']
try:
    for city, pro_com in CITIES.items():
        count = 0.0
        
        for city_section in quarters[pro_com]:
            count += 1.0
            print(pro_com, city_section, count/len(quarters[pro_com]))
            
            #
            # Mixed land use
            #
            
            #LUM5_single
            sql = text("SELECT COALESCE(SUM(CASE WHEN code IN('11100', '11210', '11220', '11230', '11240', '11300') THEN ST_Area(v.geom) END), 0) as res_sum, \
COALESCE(SUM(CASE WHEN code IN('12100') THEN ST_Area(v.geom) END), 0) as com_sum,  \
COALESCE(SUM(CASE WHEN code IN('14100', '14200', '50000') THEN ST_Area(v.geom) END), 0) as oth_sum \
from atlas_sezioni v \
where v.ace = :neigh_id and v.pro_com = :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            LUM5_single_stats = np.array(result.fetchone())
            LUM5_single = stats.entropy(LUM5_single_stats)/np.log(len(LUM5_single_stats))
            results_df.loc[(city_section, pro_com), "LUM5_single"] = LUM5_single
            
            assert(LUM5_single <= 1.0 and LUM5_single >= 0)
            
            # RNR_nres_area
            sql = text('select SUM(res_area::real), SUM(nonres_area::real) \
from ( \
select SUM("E3"::real)/SUM("E2"::real) * ST_Area(geom) as res_area, \
(SUM("E2"::real)-SUM("E3"::real))/SUM("E2"::real) * ST_Area(geom) as nonres_area, sez, ace \
from istat_indicatori i  \
inner join census_areas on ace = i."ACE" and pro_com=i."PROCOM" and sez=i."NSEZ" \
where pro_com = :city_code \
group by geom, sez, ace \
having SUM("E2"::real) > 0 \
) foo \
where ace = :neigh_id')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            RNR_nres = result.fetchone()
            RNR_nres = 1- math.fabs((RNR_nres[0]-RNR_nres[1]) / (RNR_nres[0] + RNR_nres[1]))
            results_df.loc[(city_section, pro_com), "RNR_nres"] = RNR_nres
            
            assert(RNR_nres <= 1.0 and RNR_nres >= 0)
            
            
            # nig_rat_daily
            sql = text("SELECT COALESCE(Count(distinct CASE WHEN category IN ('NightLife', 'Art-night') THEN venueid END), 0) as night_num,  \
COALESCE(Count(distinct CASE WHEN category IN ('daily-use', 'non-daily-use', 'Organized activity', 'Services', 'Schools', 'Offices', 'Cultural', 'Food') THEN venueid END), 0) as daily_num \
from foursquare_venues v  \
inner join istat_sezioni s on ST_Within(v.geom, s.geom) \
where s.ace = :neigh_id and s.pro_com = :city_code", (city_section, pro_com))
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            result = result.fetchone()
            nig_tot_places = float(result[0])
            nig_rat_daily = 0
            if result[1] != 0:
                nig_rat_daily = float(result[0])/float(result[1])
            results_df.loc[(city_section, pro_com), "nig_rat_daily"] = nig_rat_daily
            
            
            
            # hType_mix
            sql = text('select SUM("E17"::real), SUM("E18"::real), SUM("E19"::real), SUM("E20"::real) from istat_indicatori where "ACE" = :neigh_id AND "PROCOM" = :city_code')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            hType_mix_stats = np.array(result.fetchone())
            hType_mix = stats.entropy(hType_mix_stats)/np.log(len(hType_mix_stats))
            
            assert(hType_mix <= 1.0 and hType_mix >= 0)
            
            #print bld_entr_interiors
            results_df.loc[(city_section, pro_com), "hType_mix"] = hType_mix
            results_df.loc[(city_section, pro_com), "hType_mix2"] = np.average(htypes_w, weights=hType_mix_stats)
            results_df.loc[(city_section, pro_com), "hType_mix3"] = np.average(htypes_w[:3], weights=hType_mix_stats[:3])
            
            # mdist_nres_daily
            sql = text("SELECT AVG(ST_Distance_Sphere(ext.geom, (SELECT a.geom \
    from foursquare_venues as a \
    where a.category IN('daily-use', 'Food') \
    ORDER BY \
    a.geom <-> \
    (ext.geom) \
    LIMIT 1 \
    ) ) ) \
    from atlas_sezioni as ext \
    where ext.code IN('11100', '11210', '11220', '11230', '11240', '11300', '12100', '14200') and ext.ace = :neigh_id and ext.pro_com= :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            mdist_nres_ndaily = result.fetchone()[0]
            results_df.loc[(city_section, pro_com), "mdist_nres_daily"] = mdist_nres_ndaily
            
            # ratio_daily_nondaily
            sql = text("SELECT COALESCE(Count(distinct CASE WHEN category IN ('Food','daily-use') THEN venueid END), 0) as daily_num,  \
COALESCE(Count(distinct CASE WHEN category IN ('non-daily-use', 'Organized activity') THEN venueid END), 0) as nondaily_num \
from foursquare_venues v  \
inner join istat_sezioni s on ST_Within(v.geom, s.geom) \
where s.ace = :neigh_id and s.pro_com = :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            comm = result.fetchone()
            ratio_daily_nondaily = 0
            if (comm[1] + comm[0]) > 0:
                ratio_daily_nondaily = float(comm[1])/float(comm[1] + comm[0])
            results_df.loc[(city_section, pro_com), "ratio_daily_nondaily"] = ratio_daily_nondaily
            
            
            
            #
            # Contact opportunities
            #
            
            # total area of the section
            sql = text("select ST_Area(geom::geography)::real from istat_sezioni where ace = :neigh_id AND pro_com= :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            total_area = result.fetchone()[0]
            results_df.loc[(city_section, pro_com), "area"] = total_area
            
            
            # total area without rivers and parks
            sql = text("Select area from atlas_area_novac a \
WHERE a.ace = :neigh_id and a.pro_com=:city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            area_to_subtract = result.fetchone()
            total_area_without_parksrivers = total_area
            if area_to_subtract:
                total_area_without_parksrivers = total_area - area_to_subtract[0]
            
                assert(total_area_without_parksrivers > 0)
                
            results_df.loc[(city_section, pro_com), "area_filtr"] = total_area_without_parksrivers
            
            
            results_df.loc[(city_section, pro_com), "nig_rat_daily3"] = float(nig_tot_places)/total_area_without_parksrivers
            results_df.loc[(city_section, pro_com), "den_nres_daily"] = float(comm[0])/total_area_without_parksrivers
            results_df.loc[(city_section, pro_com), "den_nres_non-daily"] = float(comm[1])/total_area_without_parksrivers
            
            
            sql = text("select count(distinct v.venueid) as num_community_places \
from foursquare_venues v \
inner join istat_sezioni s on ST_Within(v.geom, s.geom) \
and s.ace = :neigh_id and s.pro_com = :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            tot_places = float(result.fetchone()[0])
            
            
            # num_community_places
            sql = text("select count(distinct v.venueid) as num_community_places \
from foursquare_venues v \
inner join istat_sezioni s on ST_Within(v.geom, s.geom) \
where v.category IN('Parks and outside', 'Organized activity', 'Food', 'Shops') \
and s.ace = :neigh_id and s.pro_com = :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            num_community_places = int(result.fetchone()[0])
            results_df.loc[(city_section, pro_com), "num_community_places"] = num_community_places
            results_df.loc[(city_section, pro_com), "num_community_places_poi"] = 0
            if tot_places > 0:
                results_df.loc[(city_section, pro_com), "num_community_places_poi"] = float(num_community_places)/tot_places
            
            
            # sphi
            sql = text("Select ST_AsText(g) \
from ( \
SELECT (ST_Dump(ST_Difference(sezione.geom, roads_buffered.geom))).geom as g  \
from istat_sezioni as sezione, roads_buffered \
where roads_buffered.ace = sezione.ace AND roads_buffered.pro_com = sezione.pro_com AND sezione.ace = :neigh_id AND sezione.pro_com= :city_code \
) as foo \
where ST_Area(g::geography) > 3000;")
            results = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            sphi_stats = []
            for r in results:
                p = to_shape(WKTElement(r[0]))
                pts = np.array(p.exterior.coords)
                # Stable version
                mean_pts = np.mean(pts,0)
                pts -= mean_pts
                result = make_circle(pts)
                coo = (result[0] + mean_pts[0], result[1] + mean_pts[1], result[2])
                sphi = float(p.area / (math.pi*coo[2]**2))
                assert (sphi <= 1.0)
                sphi_stats.append(sphi)
            sphi = np.array(sphi_stats).mean()
            results_df.loc[(city_section, pro_com), "sphi"] = sphi
            #print "sphi finito"
            
            # avg_block_area
            sql = text("Select area::real \
from ( \
SELECT ST_Area( \
(ST_Dump(ST_Difference(sezione.geom, roads_buffered.geom))).geom::geography) as area \
from istat_sezioni as sezione, roads_buffered \
where roads_buffered.ace = sezione.ace AND roads_buffered.pro_com = sezione.pro_com AND sezione.ace = :neigh_id AND sezione.pro_com= :city_code \
) as foo \
where area > 3000;")
            results = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            areas = []
            for r in results:
                areas.append(r[0])
            #avg_block_area = np.mean(stats.boxcox(areas, lmbda=(-0.32)))
            avg_block_area = np.mean(np.log(areas))
            #print avg_block_area, rat_block_area
            results_df.loc[(city_section, pro_com), "avg_block_area"] = avg_block_area
            
            
            # 2-ways intersections
            sql = text("SELECT COUNT(*) as num_intersect FROM roads_2ways_sezioni \
WHERE ace = :neigh_id AND pro_com = :city_code") 
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            num_intersect = int(result.fetchone()[0])
            
            # roundabout 2-ways intersections
            sql = text("select num, numd from roads_roundabout_sezioni \
WHERE ace = :neigh_id AND pro_com = :city_code")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            for r in result.fetchall():
                num_intersect += int((r[1] +1))
            
            results_df.loc[(city_section, pro_com), "num_intersect"] = num_intersect
            
            #
            # Aged buildings and enterprises
            #
            
            # bld_entr_age
            sql = text('select SUM("E8"::real), SUM("E9"::real), SUM("E10"::real), SUM("E11"::real), SUM("E12"::real), SUM("E13"::real), SUM("E14"::real), (SUM("E15"::real)+SUM("E16"::real)) from istat_indicatori where "ACE" = :neigh_id AND "PROCOM" = :city_code')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            buildings_age_stats = np.array(result.fetchone())
            bld_entr_age = stats.entropy(buildings_age_stats)/np.log(len(buildings_age_stats))
            
            assert(bld_entr_age <= 1.0 and bld_entr_age >= 0)
            
            #print bld_entr_age
            results_df.loc[(city_section, pro_com), "bld_entr_age"] = bld_entr_age
            
            #ref: http://math.stackexchange.com/questions/320441/standard-deviation-of-the-weighted-mean
            age_mean = np.average(age_mean_intervals, weights=buildings_age_stats[1:])
            age_std_w = math.sqrt(age_mean_intervals.var() * (np.sum(np.power(buildings_age_stats[1:], 2))) / (np.power(np.sum(buildings_age_stats[1:]), 2)))
            
            results_df.loc[(city_section, pro_com), "bld_avg_age"] = age_mean
            results_df.loc[(city_section, pro_com), "bld_std_age"] = age_std_w
            
            
            # bld_entr_interiors
            sql = text('select SUM("E21"::real), SUM("E22"::real), SUM("E23"::real), SUM("E24"::real), SUM("E25"::real), SUM("E26"::real) from istat_indicatori where "ACE" = :neigh_id AND "PROCOM" = :city_code')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            buildings_int_stats = np.array(result.fetchone())
            bld_entr_interiors = stats.entropy(buildings_int_stats)
            
            #print bld_entr_interiors
            results_df.loc[(city_section, pro_com), "bld_entr_interiors"] = bld_entr_interiors
            
            # enterprises_entr_size
            # 4 dimensions of companies => entropy
            sql = text('select count(*)::int \
from companies c \
inner join istat_sezioni sezione ON ST_Within(c.geom, sezione.geom) \
where dimension <> \'n.d.\' AND sezione.ace = :neigh_id and sezione.pro_com=:city_code \
group by dimension order by dimension')
            results = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            enterprises_stats = []
            for r in results:
                enterprises_stats.append(r[0])
            enterprises_entr_size = stats.entropy(enterprises_stats)
            
            #print enterprises_entr_size
            results_df.loc[(city_section, pro_com), "enterprises_entr_size"] = enterprises_entr_size
            
            
            sql = text('select AVG("ADDETTI"::real) as average_size \
from istat_industria where "NSEZ" IN(select sez::int from census_areas where ace = :neigh_id and pro_com = :city_code) \
and "PROCOM" = :city_code')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            avg_entr = result.fetchone()[0]
            
            results_df.loc[(city_section, pro_com), "enterprises_empl_size"] = avg_entr
            
            
            #
            # Density
            #

            # pop_rat_num, emp_rat_num, emp_rat_pop, bld_rat_int
            sql = text('select SUM("P1"::real)/:area as pop_rat_num, \
            SUM("E27"::real)/SUM("E3"::real) as bld_rat_int from istat_indicatori where "ACE" = :neigh_id AND "PROCOM" = :city_code')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com, area=total_area_without_parksrivers)
            pop_rat_num, bld_rat_int = result.fetchone()
            #print pop_rat_num
            assert(pop_rat_num <= 1.0)
            assert(bld_rat_int >= 1.0)
            results_df.loc[(city_section, pro_com), "pop_rat_num"] = pop_rat_num
            results_df.loc[(city_section, pro_com), "bld_rat_int"] = bld_rat_int
            
            # emp_rat_num, emp_rat_pop
            sql = text('select SUM("ADDETTI"::real)/:area as emp_rat_num \
            from istat_industria where "NSEZ" IN(select sez::int from census_areas where ace = :neigh_id and pro_com = :city_code) \
            and "PROCOM" = :city_code')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com, area=total_area_without_parksrivers)
            emp_rat_num = result.fetchone()[0]
            emp_rat_pop = pop_rat_num/emp_rat_num
            
            assert(emp_rat_num <= 1.0)
            
            results_df.loc[(city_section, pro_com), "emp_rat_pop"] = emp_rat_pop
            results_df.loc[(city_section, pro_com), "emp_rat_num"] = emp_rat_num
            
            
            # bld_rat_area
            sql = text('select sum(a.area::real)/:area as bld_rat_area from  \
(Select ST_Area(( \
CASE WHEN ST_CoveredBy(buildings_sezioni.geom, s.geom) THEN buildings_sezioni.geom ELSE ST_Multi(ST_Intersection(buildings_sezioni.geom,s.geom)) END)::geography) as area \
from buildings_sezioni \
inner join istat_sezioni s ON s.ace = buildings_sezioni.ace AND s.pro_com = buildings_sezioni.pro_com \
where buildings_sezioni.ace = :neigh_id and buildings_sezioni.pro_com=:city_code) as a')
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com, area=total_area_without_parksrivers)
            bld_rat_area = result.fetchone()[0]
            
            # Fix precision errors
            if bld_rat_area > 1:
                bld_rat_area = 1
            
            #print bld_rat_area
            results_df.loc[(city_section, pro_com), "bld_rat_area"] = bld_rat_area
            
            
            #
            # Border vacuums
            #
            
            # bor_rat_area
            sql = text("Select COALESCE(SUM(ST_Area(ST_intersection(a.geom, s.geom)::geography)), 0) from atlas_sezioni a \
inner join istat_sezioni s ON s.ace = a.ace AND s.pro_com = a.pro_com \
WHERE a.ace = :neigh_id and a.pro_com=:city_code AND a.code IN ('12210', '50000', '12230');")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            bor_rat_area = result.fetchone()[0]
            
            sql = text("Select COALESCE(SUM(ST_Area(ST_intersection(a.geom, s.geom)::geography)), 0) from parks a \
inner join istat_sezioni s ON ST_Intersects(a.geom, s.geom) \
WHERE s.ace = :neigh_id and s.pro_com=:city_code AND a.geoarea > 100000 and ST_isvalid(a.geom)")
            result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
            parks_area = result.fetchone()[0]
            bor_rat_area = (bor_rat_area + parks_area)/total_area
            
            results_df.loc[(city_section, pro_com), "bor_rat_area"] = bor_rat_area
            
            if bor_rat_area > 1.0:
                bor_rat_area = 1.0
            
            
            vacuums = {"mdist_railways": 12230, "mdist_parks": 14100, "mdist_highways": 12210, "mdist_water": 50000, "mdist_smallparks": 3}
            for k, v in vacuums.items():
                if k in {"mdist_highways", "mdist_water"}:
                    sql = text("select avg(dist) as d_avg \
from( \
	SELECT dist, ROW_NUMBER() OVER (PARTITION BY g1 order by dist) AS r \
	from ( select a.geom as g1, ST_DistanceSphere(a.geom, b.geom) as dist \
		FROM atlas_sezioni a, \
		LATERAL ( select vacuum.geom \
			FROM atlas_sezioni vacuum \
			WHERE a.pro_com = vacuum.pro_com AND vacuum.code = ':vacuum_code' \
			ORDER BY vacuum.geom <-> a.geom \
			LIMIT 30 \
		) b \
		WHERE a.pro_com = :city_code and a.ace = :neigh_id AND a.code IN('11100', '11210', '11220', '11230', '11240', '11300', '12100', '14200') \
	) odtable  \
) x WHERE x.r = 1;")
                    result = engine.execute(sql, neigh_id=city_section, city_code=pro_com, vacuum_code=v)
                elif k == "mdist_railways":
                    sql = text("select avg(dist) as d_avg \
from( \
	SELECT dist, ROW_NUMBER() OVER (PARTITION BY g1 order by dist) AS r \
	from ( select a.geom as g1, ST_DistanceSphere(a.geom, b.geom) as dist \
		FROM atlas_sezioni a, \
		LATERAL ( select vacuum.geom \
			FROM atlas_railways vacuum \
			WHERE a.pro_com = vacuum.pro_com \
			ORDER BY vacuum.geom <-> a.geom \
			LIMIT 30 \
		) b \
		WHERE a.pro_com = :city_code and a.ace = :neigh_id AND a.code IN('11100', '11210', '11220', '11230', '11240', '11300', '12100', '14200') \
	) odtable \
) x WHERE x.r = 1;")
                    result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
                elif k == "mdist_smallparks":
                    sql = text("SELECT AVG(ST_DistanceSphere(ext.geom, (SELECT a.geom \
    from parks as a \
    where a.geoarea <= 10000 \
    ORDER BY \
    a.geom <#> \
    (ext.geom) \
    LIMIT 1 \
    ) ) ) \
    from atlas_sezioni as ext \
    where ext.code IN('11100', '11210', '11220', '11230', '11240', '11300', '12100', '14200') and ext.ace = :neigh_id and ext.pro_com = :city_code")
                    result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
                else:
                    sql = text("SELECT AVG(ST_DistanceSphere(ext.geom, (SELECT a.geom \
    from parks as a \
    where a.geoarea > 100000  and ST_isvalid(a.geom) \
    ORDER BY \
    a.geom <-> \
    (ext.geom) \
    LIMIT 1 \
    ) ) ) \
    from atlas_sezioni as ext \
    where ext.code IN('11100', '11210', '11220', '11230', '11240', '11300', '12100', '14200') and ext.ace = :neigh_id and ext.pro_com=:city_code")
                    result = engine.execute(sql, neigh_id=city_section, city_code=pro_com)
                
                results_df.loc[(city_section, pro_com), k] = result.fetchone()[0]
            
except:
    raise Exception("".join(traceback.format_exception(*sys.exc_info())))

15146 30 0.011764705882352941
4882782.768728021 1
15146 48 0.023529411764705882
1468225.217290009 1
15146 28 0.03529411764705882
2317991.566914328 1
15146 20 0.047058823529411764
566839.4661657303 1


/data/nadai/compiled/python3.6.4/lib/python3.6/site-packages/ipykernel_launcher.py:141: RuntimeWarning: Mean of empty slice.
/data/nadai/compiled/python3.6.4/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/data/nadai/compiled/python3.6.4/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


15146 62 0.058823529411764705
1531643.7663193343 1
15146 80 0.07058823529411765
2122626.45215954 1
15146 8 0.08235294117647059
1569740.7626129147 1
15146 27 0.09411764705882353
992474.0 1.0
15146 26 0.10588235294117647
911739.8713214269 1
15146 25 0.11764705882352941
744470.7072395597 1
15146 82 0.12941176470588237
1069624.488429317 1
15146 66 0.1411764705882353
1702065.287085221 1
15146 33 0.15294117647058825
1872249.6128425098 1
15146 50 0.16470588235294117
2214643.3773667812 1
15146 17 0.17647058823529413
798226.141109472 1
15146 69 0.18823529411764706
4485666.189943539 1
15146 76 0.2
3519366.679043402 1
15146 31 0.21176470588235294
1708071.136902785 1
15146 51 0.2235294117647059
1627942.981505203 1
15146 57 0.23529411764705882
732231.0 1
15146 11 0.24705882352941178
895630.5176194198 1
15146 39 0.25882352941176473
3767228.281249307 1
15146 67 0.27058823529411763
1250665.011401501 1
15146 74 0.2823529411764706
1528974.6870703143 1
15146 64 0.29411764705882354
1978631.674416746 1
151

In [25]:
results_df.to_csv('generated_files/computed_metrics.csv')